## Введение в обработку естественного языка

Домашнее задание №14

Урок 14. Transfer learning

*Формат именования файла домашней работы: FIO_NLP_HW_N.ipynb, где N - номер домашнего задания*

**

Задание

1. взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes обучить модель GPT для генерации своих цитат


2. взять новостные данные из https://github.com/natasha/corus load_lenta2 нам понадобиться сам текст и заголовок обучить модель T5/ или GPT для генерации заголовков для статей

In [1]:
import pandas as pd
import random

In [2]:
!pip install transformers sentencepiece --quiet

## Задание 1

In [3]:
from google.colab import files # чтобы загружать файлы в облако через проводник 

In [4]:
uploaded = files.upload() #просто запускаем и выбираем файл в проводнике

#↓ строка которая активирует наш API токен
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [5]:
!kaggle datasets download mrapplexz/bashim-quotes

bashim-quotes.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!unzip bashim-quotes.zip

Archive:  bashim-quotes.zip
replace dataset.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset.jsonl           


In [7]:
DATA_PATH = '/content/dataset.jsonl'

with open(DATA_PATH) as f:
     df = pd.read_json(DATA_PATH, lines=True).set_index('id')
df.head(3)

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз..."


In [8]:
import re

def clear_text(text):
    clr_text = re.sub(r"<.*?>", " ", text).lower()
    clr_text = summary = re.sub(r"\s", " ", clr_text)
    return clr_text

In [9]:
df["clear_text"] = df["text"].apply(lambda x: clear_text(x))
df

,date,rating,text,clear_text
id,,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они...","ppdv, все юниксы очень дружелюбны.. они прос..."
2,2004-08-30 11:25:00+00:00,25105.0,<томатик_рад> а ты не чувствуешь красоту мира?...,а ты не чувствуешь красоту мира? честно го...
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаз...","""мышка, почему у тебя такие большие глаза?"" ..."
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать ...","""мальчики, вы что больные, бегать в палату к..."
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с ...,мы - как разработчики - живём с субейзом под...
...,...,...,...,...
463644,2020-11-26 06:12:00+00:00,698.0,"xxx: угадайте не гугля, что такое жопиздан!\ny...","xxx: угадайте не гугля, что такое жопиздан! yy..."
463645,2020-11-26 06:12:00+00:00,816.0,"xxx:\nПосетила шальная мысль заняться собой, ж...","xxx: посетила шальная мысль заняться собой, жи..."
463646,2020-11-26 06:46:00+00:00,88.0,#всебожьяроса\nxxx: Судьба айтишников вообще н...,#всебожьяроса xxx: судьба айтишников вообще не...


In [10]:
data = df.loc[:, 'clear_text']

In [11]:
data

id
1           ppdv, все юниксы очень дружелюбны.. они прос...
2           а ты не чувствуешь красоту мира?   честно го...
3           "мышка, почему у тебя такие большие глаза?" ...
4           "мальчики, вы что больные, бегать в палату к...
5           мы - как разработчики - живём с субейзом под...
                                ...                        
463644    xxx: угадайте не гугля, что такое жопиздан! yy...
463645    xxx: посетила шальная мысль заняться собой, жи...
463646    #всебожьяроса xxx: судьба айтишников вообще не...
463647    прибывшие на место правоохранители установили,...
463648    ххх: вот ребята из английской фирмы tvr (давны...
Name: clear_text, Length: 81497, dtype: object

In [12]:
text=df['clear_text']

In [13]:
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        data += summary + "  "
    f.write(data)

In [14]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train, 'train_dataset.txt')
build_text_files(test, 'test_dataset.txt')

In [15]:
str(len(train))

'69272'

In [16]:
str(len(test))

'12225'

In [17]:
train

id
425614    xxx: помнишь ты мне смешные переводы присылал?...
457165    ххх: звонила мама вали, спрашивала, что такое ...
448546    vitus_wagner: увидев на рбк сообщение крупными...
446198    новости: работающим пенсионерам с 1 августа пр...
391159    #linux   вы линуксойды, все тупые уроды! кроме...
                                ...                        
422475    various artist: жаль, что писательским даром м...
421814    xxx: первое правило при болезни - никогда не с...
405536    время полдень. звоню другу (он недавно устроил...
429690    маме через пару месяцев восемьдесят лет. освои...
232046    arckest: зачет по физике и эволюции звезд. нич...
Name: clear_text, Length: 69272, dtype: object

In [18]:
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM

import torch
device = torch.device("cuda")

In [19]:
# Загружаем large GPT3, которая основана на GPT2
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
#model_name = 'Grossmend/rudialogpt3_medium_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [21]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
# Tuning model

training_args = TrainingArguments(

    "phrase",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    learning_rate=1e-5,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    
    )

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [24]:
# trainer.train()

Значительный минус  NLP - длительное время  обучения, только после которого ты понимаешь, что ошибся где-либо при обработке данных ;)
Сейчас же я просто остановил обучение, т.к. ждать 5 часов конца обучения одной эпохи боюсь colab мне не даст и выкинет раньше из-за истечения времени пользования ресурсами gpu

In [25]:
sep = '\n***\n'  # Признак того, что твит закончился и нужногенерировать ещё один твит
# sep = '\n27479153	Sandy_mustache	2021-02-18 16:44:00	'


# Так как мы постоянно сэмплируем разные твиты,
# мы будем постоянно получать разное распределение
prefix = sep.join([''] + random.sample(list(text), k=5) + [''])

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]  # '***' - токен который будет оканчивать твит

# выводим то, что мы передаём на вход
print(prefix)


***
dago: у нас в офисе новое растение grey: еще одного бухгалтера взяли?
***
--> lizaveta has joined this channel (679@62.105.15.72).   эй ей   здраствуйте лизочка   скажите, какого числа вы родились?   17   какого месяца?   06   какого года?   1987   какого хуя?   ??????????????????????
***
функция wolf() в полнолуние void на луну!
***
vision: в общем, если я не могу заставить организм спать, ща пойду мыть посуду vision: а то там небось уже тараканы в царя горы играют
***
xxx: знакомые рссказывали. мужик садился в машину на гаражах. куда-то запропастился навесной замок, он второпях просто замотал дверь проволокой и уехал. ночью воры вскрыли несколько гаражей - три влево и три вправо от гаража этого мужика. его гараж - с проволокой вместо замка - не тронули, а на дверях красовалась надпись баллончиком "суперзамки не взламываем"
***



In [26]:
# Генерируем какой то осмысленный текст похожий на твит.

size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False,  # вкл/выкл режим выдачи нескольких вариантов д.б. ещё один параметр
    max_length=size+128, 
    repetition_penalty=4.2,  # штраф за повторы одинарных токенов
    temperature=0.7,  # температура
    num_beams=10,  # Строим дерево глубины 10
    # no_repeat_ngram_size=3  # ! тройки подряд идущих токенов не должны повторяться (3 и меньше токенов не должны повторяться)
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


uri_loskutov: вот что бывает, когда едешь со скоростью 100 км/ч по городу...
***
sergey_petrovich: как же мне надоели эти тупые блондинки!!!!!
***
maximilianus: ААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААААА


In [27]:
# Генерируем какой то осмысленный текст похожий на твит.

size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False,  # вкл/выкл режим выдачи нескольких вариантов д.б. ещё один параметр
    max_length=size+128, 
    repetition_penalty=4.2,  # штраф за повторы одинарных токенов
    temperature=0.7,  # температура
    num_beams=5,  # Строим дерево глубины 10
    # no_repeat_ngram_size=3  # ! тройки подряд идущих токенов не должны повторяться (3 и меньше токенов не должны повторяться)
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


oung_cat: даааа... это ж надо было додуматься до такой хуйни!!!
***
sergey_petrovich: как же мне нравится эта фраза из анекдота про блондинку и брюнетку :)))))
***
maximilianus: ооооойййййййййййййййййййййййййййййййййййййййййййййййййййййййййййй


Мда, неожиданно, но интересно, и с матерком, и про блондинок

## Задание 2

In [28]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.5 MB/s eta 0:00:00


In [29]:
from datasets import load_dataset

dataset_train = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'train[:10%]')
dataset_test = load_dataset('IlyaGusev/gazeta', revision="v1.0", split= 'test[:10%]')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/52400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5265 [00:00<?, ? examples/s]

Dataset gazeta downloaded and prepared to /root/.cache/huggingface/datasets/IlyaGusev___gazeta/default/1.0.0/ef9349c3c0f3112ca4036520d76c4bc1b8a79d30bc29643c6cae5a094d44e457. Subsequent calls will reuse this data.


In [30]:
dataset_train

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 5240
})

In [31]:
dataset_test

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 577
})

In [32]:
dataset_test['summary'][0]

'В NASA назвали четыре миссии в дальний космос, которые в этом десятилетии могут быть запущены американцами. Среди них — две миссии по изучению Венеры, полет к спутнику Юпитера и экспедиция к Тритону, спутнику Нептуна.'

In [33]:
dataset_test['title'][0]

'Венера, Ио или Тритон: куда полетит NASA'

In [34]:
model_name = "IlyaGusev/rut5_base_sum_gazeta"

In [35]:
def len_tok(text):
    return len(text.split())

In [36]:
max_len_sum, max_len_tl = max(map(len_tok, dataset_train['summary'])), max(map(len_tok, dataset_train['title']))
max_len_sum, max_len_tl

(75, 18)

In [37]:
max_len_sum, max_len_tl = 60, 15

In [38]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    tokenized_input = tokenizer(batch['summary'], padding='max_length', truncation=True, max_length=max_len_sum)
    tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
dataset_test.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/5240 [00:00<?, ? examples/s]

Map:   0%|          | 0/577 [00:00<?, ? examples/s]

In [39]:
dataset_train.save_to_disk('gazeta/train')
dataset_test.save_to_disk('gazeta/test')

Saving the dataset (0/1 shards):   0%|          | 0/5240 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/577 [00:00<?, ? examples/s]

In [40]:
!ls

bashim-quotes.zip					gazeta
cached_lm_GPT2TokenizerFast_128_test_dataset.txt	phrase
cached_lm_GPT2TokenizerFast_128_test_dataset.txt.lock	sample_data
cached_lm_GPT2TokenizerFast_128_train_dataset.txt	test_dataset.txt
cached_lm_GPT2TokenizerFast_128_train_dataset.txt.lock	train_dataset.txt
dataset.jsonl


In [41]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model = T5ForConditionalGeneration.from_pretrained(model_name)

In [42]:
output_dir = 'gazeta/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.00001,
    evaluation_strategy='steps', # Run evaluation every eval_steps
    save_steps=1000, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_gazeta', # Wandb run name
    logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
)

In [43]:
%%time

# Обучение. У нас 10 эпох.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
500,8.497300,3.362565
1000,4.612700,3.160403
1500,2.774000,3.007473
2000,2.584600,2.916190
2500,2.500700,2.877698
3000,2.462000,2.857345
3500,2.421400,2.835496
4000,2.443500,2.818262
4500,2.391400,2.807277
5000,2.402900,2.796540


CPU times: user 24min 20s, sys: 1min 55s, total: 26min 16s
Wall time: 27min 46s


TrainOutput(global_step=6550, training_loss=3.084252645477994, metrics={'train_runtime': 1666.0887, 'train_samples_per_second': 31.451, 'train_steps_per_second': 3.931, 'total_flos': 4174008606720000.0, 'train_loss': 3.084252645477994, 'epoch': 10.0})

In [44]:
trainer.save_model(output_dir + '/model')

In [45]:
INX = 100
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

SUMMARY: | Российские фрегаты «Адмирал Макаров» и «Адмирал Григорович» с крылатыми ракетами «Калибр-НК» проходят через турецкие проливы Босфор и Дарданеллы в направлении Средиземного моря. Ранее эти корабли уже участвовали в военной операции РФ в Сирии и теперь направляются туда на фоне обострившейся ситуации в Идлибе.
TITLE: | Снова в Сирию: российские фрегаты идут в Средиземное море


In [46]:
device = "cuda"

In [47]:
import torch

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=10,
        temperature = 1.3,  
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



output:
«Адмирал Григорович» и «Калибр-НК


In [48]:
INX = 11
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=10,
        temperature = 1.3,  
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

SUMMARY: | Юридическая компания из США подала в суд на Apple и Samsung за выпуск смартфонов, чье излучение якобы превышает установленные законом нормы. По версии истца, вендоры подвергают здоровье своих клиентов опасности. Между тем Федеральная комиссия по связи США провела собственное расследование уровня излучения популярных гаджетов и поделилась его результатами.
TITLE: | Опасное излучение: на Apple и Samsung подали в суд

output:
Смартфоны: Apple и Samsung подвергают опасность


In [49]:
INX = 15
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=10,
        temperature = 1.3,  
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

SUMMARY: | Для борьбы с с недобросовестными АЗС, недоливающими топливо, будут использоваться специальные приборы, а также проверки программного обеспечения. В этом вопросе Росстандарт намерен использовать опыт, полученный в сфере регулирования работы игровых автоматов.
TITLE: | Борьба с недоливом: заправки проверят скрытым прибором

output:
Недобросовестные АЗС будут проверять


In [50]:
INX = 20
print("SUMMARY: | {}".format(dataset_test['summary'][INX]))
print("TITLE: | {}".format(dataset_test['title'][INX]))

input_text = dataset_test['summary'][INX]

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=10,
        temperature = 1.3,  
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2  # количество повторов n-грамм > 2 запрещено.
    )

    # Параметры подбираются эксперементально
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

SUMMARY: | Компания Apple намерена выпустить «бюджетный» смартфон iPhone SE 2 в марте следующего года. Этот гаджет станет преемником iPhone SE и обойдется покупателю не дороже $399 за базовую модель.
TITLE: | Новый дешевый iPhone: что готовит Apple

output:
Apple намерена выпустить iPhone SE 2


Вполне хорошие получаются заголовки. Мне нравится ;)